In [ ]:
pip install SpeechRecognition


In [ ]:
pip install twilio


In [ ]:
import speech_recognition as sr
from twilio.rest import Client

# Emergency keywords
emergency_keywords = ['help', 'danger', 'emergency']

# Twilio setup (replace these with your actual credentials)
account_sid = 'AC5220d581d44eba2bd74bb029595c0e37'
auth_token = '9082860630ea4130577730f13cd36eb4'
twilio_number = '+19842797413'         # Your Twilio number
to_number = '+918600893816'         # Receiver's phone number (must be verified if you're using a trial account)

client = Client(account_sid, auth_token)

# Function to make an emergency call
def make_emergency_call(keyword):
    custom_message = f"This is an emergency alert from the AI voice detection system. The keyword {keyword.upper()} was detected. Please respond immediately."

    twiml_message = f"""
    <Response>
        <Say voice="alice">{custom_message}</Say>
    </Response>
    """

    call = client.calls.create(
        twiml=twiml_message,
        to=to_number,
        from_=twilio_number
    )
    print("📞 Call initiated successfully. Call SID:", call.sid)

# Voice recognition
recognizer = sr.Recognizer()

with sr.Microphone() as source:
    print("🎙 Speak now...")
    audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio).lower()
        print("✅ You said:", text)

        for keyword in emergency_keywords:
            if keyword in text:
                print("🚨 Emergency Detected: Keyword -", keyword.upper())
                make_emergency_call(keyword)
                break
        else:
            print("👍 No emergency keywords found.")

    except sr.UnknownValueError:
        print("❌ Could not understand audio.")
    except sr.RequestError as e:
        print("❌ Speech recognition API error:", e)


🎙 Speak now...
✅ You said: help help help
🚨 Emergency Detected: Keyword - HELP
📞 Call initiated successfully. Call SID: CA7258e79307a9945a192be55f93bc93a0


In [25]:
from flask import Flask, render_template
import speech_recognition as sr
from twilio.rest import Client

app = Flask(__name__)

# Emergency keywords
emergency_keywords = ['help', 'danger', 'emergency']

# Twilio setup
account_sid = 'AC5220d581d44eba2bd74bb029595c0e37'
auth_token = '9082860630ea4130577730f13cd36eb4'
twilio_number = '+19842797413'
to_number = '+918600893816'

client = Client(account_sid, auth_token)

# Function to make a call
def make_emergency_call(keyword):
    custom_message = f"This is an emergency alert from the AI voice detection system. The keyword {keyword.upper()} was detected. Please respond immediately."
    twiml_message = f"""
    <Response>
        <Say voice="alice">{custom_message}</Say>
    </Response>
    """
    call = client.calls.create(
        twiml=twiml_message,
        to=to_number,
        from_=twilio_number
    )
    print("📞 Call initiated successfully. Call SID:", call.sid)

@app.route('/')
def index():
    return render_template('alertzone.html')  # HTML UI with trigger button

@app.route('/trigger-alert')
def trigger_alert():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎙 Listening...")
        audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio).lower()
        print("✅ You said:", text)

        for keyword in emergency_keywords:
            if keyword in text:
                print("🚨 Emergency Detected:", keyword.upper())
                make_emergency_call(keyword)
                return f"🚨 Emergency keyword '{keyword.upper()}' detected. Call initiated!"
        return "✅ No emergency keywords found."

    except sr.UnknownValueError:
        return "❌ Could not understand audio."
    except sr.RequestError as e:
        return f"❌ Speech Recognition error: {e}"

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1